<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#For-the-4-different-pre­processing-specifications,-the-dimension-of-word-count-matrix" data-toc-modified-id="For-the-4-different-pre­processing-specifications,-the-dimension-of-word-count-matrix-1">For the 4 different pre­processing specifications, the dimension of word count matrix</a></span></li><li><span><a href="#for-the-4-different-pre-processing-specifications,-report-the-top-10-terms-with-the-“strongest”-coefficients" data-toc-modified-id="for-the-4-different-pre-processing-specifications,-report-the-top-10-terms-with-the-“strongest”-coefficients-2">for the 4 different pre-processing specifications, report the top 10 terms with the “strongest” coefficients</a></span></li><li><span><a href="#define-the-TF-and-IDF-you-want-to-use-for-this-dataset." data-toc-modified-id="define-the-TF-and-IDF-you-want-to-use-for-this-dataset.-3">define the TF and IDF you want to use for this dataset.</a></span></li><li><span><a href="#for-the-4-different-pre-processing-specifications,-the-dimension-of-word-matrix" data-toc-modified-id="for-the-4-different-pre-processing-specifications,-the-dimension-of-word-matrix-4">for the 4 different pre-processing specifications, the dimension of word matrix</a></span></li><li><span><a href="#report-the-top-10-terms-with-the-“strongest”-coefficients-again" data-toc-modified-id="report-the-top-10-terms-with-the-“strongest”-coefficients-again-5">report the top 10 terms with the “strongest” coefficients again</a></span></li><li><span><a href="#comment-on-the-difference-between-with-and-without-applying-TF-IDF-when-predicting-the-job-title-using-these-different-features-in-a-paragraph." data-toc-modified-id="comment-on-the-difference-between-with-and-without-applying-TF-IDF-when-predicting-the-job-title-using-these-different-features-in-a-paragraph.-6">comment on the difference between with and without applying TF-IDF when predicting the job title using these different features in a paragraph.</a></span></li></ul></div>

In [1]:
import json

from collections import Counter
import pandas as pd
import numpy as np

from nltk.tokenize import TreebankWordTokenizer, ToktokTokenizer

from nltk.stem import PorterStemmer
from nltk.stem.snowball import EnglishStemmer


from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

import re # to remove hyperlink

In [2]:
indeed = json.load(open('indeed_title_job_desc.json', 'r'))
job_key = list(indeed.keys())

obtain the word counts for each job description

In [3]:
title = []
for key in job_key:
    for job_des in indeed[key]:
        title.append(key)


In [4]:
def pre_process(Tokenizer, Stemmer):
    data=[]
    for key in job_key:
        for job_des in indeed[key]:
            # remove hyper link
            job_des = re.sub(r'http\S+', '', job_des, flags=re.MULTILINE) 
            # lower all the 
            # tokenize the sentense
            toks = Tokenizer.tokenize(job_des)
            # Stem the tokens 
            job_des = [Stemmer.stem(tok) for tok in toks]
            # combine every loop result
            data.append(job_des)
    counts = [Counter(datum) for datum in data]
    # frequenct matrix
    df = pd.DataFrame(counts)
    return df
    

word count matrix for each pre-processing specification

In [12]:
Treebank_Porter = pre_process(TreebankWordTokenizer(), PorterStemmer())

In [13]:
Treebank_Snow = pre_process(TreebankWordTokenizer(), EnglishStemmer())

In [14]:
Tokto_Porter =  pre_process(ToktokTokenizer(), PorterStemmer())

In [15]:
Toktok_Snow = pre_process(ToktokTokenizer(), EnglishStemmer())

## For the 4 different pre­processing specifications, the dimension of word count matrix
- Using TreebankWordTokenizer, PorterStemmer generate 735 * 10981 word count matrix
- Using TreebankWordTokenizer, SnowballStemmer (English Stemmer) generate 735 * 10874 word count matrix
- Using ToktokTokenizer, PorterStemmer generate 735 * 11000 word count matrix
- Using ToktokTokenizer, SnowballStemmer (English Stemmer) generate 735 * 10901 word count matrix

In [16]:
Treebank_Porter.shape

(735, 10981)

In [17]:
Treebank_Snow.shape

(735, 10874)

In [18]:
Tokto_Porter.shape

(735, 11000)

In [19]:
Toktok_Snow.shape

(735, 10901)

logistic regression

In [20]:
def top10(count_matrix):
    X = count_matrix.fillna(0)
    y = np.array(title)
    logit = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    logit.fit(X, y)
    coef_matrix = pd.DataFrame(data = logit.coef_).transpose()
    coef_matrix.index = count_matrix.columns
    coef_matrix.columns = job_key
    coef_matrix = coef_matrix.assign(strength=lambda x:abs(x[job_key[0]])+ abs(x[job_key[1]])+
                         abs(x[job_key[2]])+abs(x[job_key[3]])+abs(x[job_key[4]]))
    coef_matrix = coef_matrix.sort_values(by=['strength'])
    top10_result = list(coef_matrix.iloc[0:10,].index)   
    return top10_result

## for the 4 different pre-processing specifications, report the top 10 terms with the “strongest” coefficients

In [21]:
Treebank_Porter_10 = top10(Treebank_Porter)
Treebank_Snow_10 = top10(Treebank_Snow)
Tokto_Porter_10 = top10(Tokto_Porter)
Toktok_Snow_10 = top10(Toktok_Snow)

/anaconda/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda/envs/nlp/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [22]:
print('Top 10 terms for TreebankWordTokenizer, PorterStemmer:', *Treebank_Porter_10, sep='\n- ')

Top 10 terms for TreebankWordTokenizer, PorterStemmer:
- usage.
- dreams.
- technology-rel
- bachelors/mast
- expansion.
- potential.
- consultingleadership
- overnight
- replac
- EE


In [23]:
print('Top 10 terms for TreebankWordTokenizer, SnowballStemmer (English Stemmer):', *Treebank_Snow_10, sep='\n- ')

Top 10 terms for TreebankWordTokenizer, SnowballStemmer (English Stemmer):
- expansion.
- dreams.
- usage.
- bachelors/mast
- potential.
- technology-rel
- building/manag
- experienceproduct
- confidence.
- never-been-done-befor


In [24]:
print('Top 10 terms for  ToktokTokenizer, PorterStemmer:', *Tokto_Porter_10, sep='\n- ')

Top 10 terms for  ToktokTokenizer, PorterStemmer:
- expansion.
- usage.
- potential.
- technology-rel
- bachelors/mast
- dreams.
- consultingleadership
- signature.
- mi
- building/manag


In [25]:
print('Top 10 terms for ToktokTokenizer, SnowballStemmer (English Stemmer):', *Toktok_Snow_10, sep='\n- ')

Top 10 terms for ToktokTokenizer, SnowballStemmer (English Stemmer):
- dreams.
- technology-rel
- usage.
- potential.
- expansion.
- bachelors/mast
- developer-readi
- confidence.
- never-been-done-befor
- mis


## define the TF and IDF you want to use for this dataset.

- Term Frequenct (TF): the number of times a word $i$ occurs in a job description $d_{j}$ divide by the total counts of all words $k$ in a job description.

- Inverse document frequency (IDF): the number N of job description divide by the number of job description $D$ containing a certain word, and take logarithm of the number .

\begin{equation*}
\text{TF}_{ij} = \big(\frac{n_{ij}}{\sum n_{k,j}}\big) + 1
\end{equation*}



\begin{equation*}
\text{IDF}_{i} = \lg\big(\frac{D}{j: t_{i}\in d_{j}}\big)
\end{equation*}


\begin{equation*}
\text{TF- IDF} = \text{TF} \times \text{IDF}
\end{equation*}





In [29]:
def tfidf_docs(Tokenizer, Stemmer):
    docs=[]
    for key in job_key:
        for job_des in indeed[key]:
            job_des = re.sub(r'http\S+', '', job_des, flags=re.MULTILINE)  # remove hyper link
            # tokenize the sentense
            toks = Tokenizer.tokenize(job_des)
            # Stem the tokens 
            job_des = [Stemmer.stem(tok) for tok in toks]
            # combine every loop result
            docs.append(job_des)
    tfidf = TfidfVectorizer(analyzer='word',tokenizer=lambda x : x ,preprocessor=lambda x : x, token_pattern=None)
    vect = tfidf.fit_transform(docs)
    X = vect
    y = np.array(title)
    logit = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    logit.fit(X, y)
    coef_matrix = pd.DataFrame(data = logit.coef_).transpose()
    coef_matrix.index = tfidf.vocabulary_
    coef_matrix.columns = job_key
    coef_matrix = coef_matrix.assign(strength=lambda x:abs(x[job_key[0]])+ abs(x[job_key[1]])+
                         abs(x[job_key[2]])+abs(x[job_key[3]])+abs(x[job_key[4]]))
    coef_matrix = coef_matrix.sort_values(by=['strength'])
    top10_result = list(coef_matrix.iloc[0:10,].index)   
    return vect.shape, top10_result
    

## for the 4 different pre-processing specifications, the dimension of word matrix

- Using TreebankWordTokenizer, PorterStemmer generate 735 * 10981 word count matrix
- Using TreebankWordTokenizer, SnowballStemmer (English Stemmer) generate 735 * 10874 word count matrix
- Using ToktokTokenizer, PorterStemmer generate 735 * 11000 word count matrix
- Using ToktokTokenizer, SnowballStemmer (English Stemmer) generate 735 * 10901 word count matrix

In [30]:
treepo=tfidf_docs(TreebankWordTokenizer(), PorterStemmer())
treeen=tfidf_docs(TreebankWordTokenizer(), EnglishStemmer())
topo=tfidf_docs(ToktokTokenizer(), PorterStemmer())
toen=tfidf_docs(ToktokTokenizer(), EnglishStemmer())

In [36]:
treepo[0]

(735, 10981)

In [38]:
treeen[0]

(735, 10874)

In [39]:
topo[0]

(735, 11000)

In [40]:
toen[0]

(735, 10901)

## report the top 10 terms with the “strongest” coefficients again

In [41]:
print('Top 10 terms for tfidf:', *treepo[1], sep='\n- ')

Top 10 terms for tfidf:
- requirements.
- bessem
- perfectlysuit
- snowflake.net.
- maintainable.
- addepar.
- /yearwork
- easy-to-us
- wholly-own
- drinksfoosbal


In [43]:
print('Top 10 terms for tfidf:', *treeen[1], sep='\n- ')

Top 10 terms for tfidf:
- scale—it
- domin
- dashboards/visu
- incred
- theater
- consumers.
- idm
- duties.
- tcp/ip
- outstand


In [44]:
print('Top 10 terms for tfidf:', *topo[1], sep='\n- ')

Top 10 terms for tfidf:
- MO
- efficiencies.
- attempt
- dip
- trello
- tools.key
- python/scala/sql
- metadata.
- stibo
- fundersclub


In [45]:
print('Top 10 terms for tfidf:', *toen[1], sep='\n- ')

Top 10 terms for tfidf:
- data-fuel
- environment.skil
- headquarters.th
- servant
- hadoopazuregoogl
- databusi
- materials.
- learningmarketingrpythonaffili
- suck
- rout


## comment on the difference between with and without applying TF-IDF when predicting the job title using these different features in a paragraph.

TF-IDF reweight words to highlights those therm which are highly frequently in a certain document but less frequently across documents, to capture the pattern of certain types of documents. 

Without TF-IDF, general words which may apply to any job description get the strongest coefficients, like bachelors, dreams, usage, potential. While using TF-IDF, words specific to data science job would pop up in terms of coefficient strength, like data, dashboard, python/scala/sql.